In [1]:
import os, re, numpy as np, pandas as pd

import requests 
from bs4 import BeautifulSoup

import folium 
from geopy.geocoders import Nominatim 

import urllib # dataya erişim
import json # data formatının dönüşümü
import pandas as pd # analizler için

In [4]:
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36"}
request = requests.get("https://www.atlasbig.com/tr/istanbul-mahalleleri-nufus-yogunlugu", headers)
soup = BeautifulSoup(request.text,'html.parser')
table = soup.findAll("table")[0]

data =  []
for tr in table.find_all("tr"):
    row = []
    for td in tr.find_all("td"):
        row.append(td.text.replace("\t", "").replace("\n", "").strip().replace(",",""))
    
    data.append(row[1:3]+row[-1:])
    
df = pd.DataFrame(data[1:], columns = ["Neighborhood", "County", "Area"]).sort_values(["County", "Neighborhood"]).reset_index(drop=True)
df["Search"] = df["County"].str.lower() + "-" + df["Neighborhood"].str.replace(" ", "-").str.replace(".", "").str.lower()
df["SearchDesc"] = df["Neighborhood"] + "," + df["County"] + ", İstanbul"
mapping = {"ç": "c", "ğ":"g", "ı":"i", "ö":"o", "ş":"s", "ü":"u"}

for k, v in mapping.items():
    df["Search"] = df["Search"].str.replace(k, v)

In [5]:
df

,Neighborhood,County,Area,Search,SearchDesc
0,Anadolu,Arnavutköy,1.5,arnavutkoy-anadolu,"Anadolu,Arnavutköy, İstanbul"
1,Arnavutköy Merkez,Arnavutköy,6.7,arnavutkoy-arnavutkoy-merkez,"Arnavutköy Merkez,Arnavutköy, İstanbul"
2,Arnavutköy Yavuzselim,Arnavutköy,2.1,arnavutkoy-arnavutkoy-yavuzselim,"Arnavutköy Yavuzselim,Arnavutköy, İstanbul"
3,Arnavutköy İmrahor,Arnavutköy,33.2,arnavutkoy-arnavutkoy-i̇mrahor,"Arnavutköy İmrahor,Arnavutköy, İstanbul"
4,Arnavutköy İslambey,Arnavutköy,1.6,arnavutkoy-arnavutkoy-i̇slambey,"Arnavutköy İslambey,Arnavutköy, İstanbul"
...,...,...,...,...,...
949,Paşa,Şişli,0.75,sisli-pasa,"Paşa,Şişli, İstanbul"
950,Teşvikiye,Şişli,0.45,sisli-tesvikiye,"Teşvikiye,Şişli, İstanbul"
951,Yayla,Şişli,0.25,sisli-yayla,"Yayla,Şişli, İstanbul"
952,İnönü,Şişli,0.18,sisli-i̇nonu,"İnönü,Şişli, İstanbul"


In [202]:
def get_neighborhoodstats(mainsoup, demosoup):
    stats = []
    #Neighborhood
    stats.append(mainsoup.select("div.header h1")[0].text) 
    #Avarage Sale Price
    stats.append(mainsoup.select("div.location-report-charts-container-content-info-data")[0].select("span")[2].text)
    #Min Sale Price
    stats.append(mainsoup.select("div.location-report-charts-container-content-info-data")[0].select("span")[3].text.replace("\n","").strip())
    #Max Sale Price
    stats.append(mainsoup.select("div.location-report-charts-container-content-info-data")[0].select("span")[5].text.replace("\n","").strip())
    #Avarage Rent Price
    stats.append(mainsoup.select("div.location-report-charts-container-content-info-data")[2].select("span")[2].text)
    #Min Rent Price
    stats.append(mainsoup.select("div.location-report-charts-container-content-info-data")[2].select("span")[3].text.replace("\n","").strip())
    #Max Rent Price
    stats.append(mainsoup.select("div.location-report-charts-container-content-info-data")[2].select("span")[5].text.replace("\n","").strip())
    #Return Perido
    stats.append(mainsoup.select("div.location-report-charts-container-content-info-data")[1].select("span")[1].text)

    #price changes
    for i in mainsoup.select("section.location-report-main-content-extra div.location-report-charts-price-content-item-top"):
        try:
            if i.select("span")[1]["style"].split("/")[-1].split(".")[0] == "price-up":
                stats.append(i.select("span")[0].text)
            else:
                stats.append("-" + i.select("span")[0].text)
        except:
            stats.append(i.select("span")[0].text)

    #demographics
    for i in demosoup.select("div.location-report-info-box p strong"):
        stats.append(i.text)
    return(stats)

In [203]:
%%time

data = []
requestheaders = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36"}

for neighborhood, county, search in zip(df["Neighborhood"], df["County"], df["Search"]):    
    print([neighborhood, county])
    mainurl = "https://www.zingat.com/"+ search +"-bolge-raporu"
    demourl = "https://www.zingat.com/"+ search +"-bolge-raporu?mode=demographics"
    mainrequest = requests.get(mainurl, requestheaders)
    demorequest = requests.get(demourl, requestheaders)
    mainsoup = BeautifulSoup(mainrequest.text,'html.parser')
    demosoup = BeautifulSoup(demorequest.text,'html.parser')
    
    if mainsoup.select("div.header h1"):
        data.append([neighborhood, county] + get_neighborhoodstats(mainsoup, demosoup))
    else:
        data.append([neighborhood, county])

['Anadolu', 'Arnavutköy']
['Arnavutköy Merkez', 'Arnavutköy']
['Arnavutköy Yavuzselim', 'Arnavutköy']
['Arnavutköy İmrahor', 'Arnavutköy']
['Arnavutköy İslambey', 'Arnavutköy']
['Baklalı', 'Arnavutköy']
['Balaban', 'Arnavutköy']
['Bolluca Merkez', 'Arnavutköy']
['Boyalık', 'Arnavutköy']
['Boğazköy Atatürk', 'Arnavutköy']
['Boğazköy İstiklal', 'Arnavutköy']
['Deliklikaya', 'Arnavutköy']
['Dursunköy', 'Arnavutköy']
['Durusu Zafer', 'Arnavutköy']
['Hacımaşlı', 'Arnavutköy']
['Hadımköy', 'Arnavutköy']
['Haraççı', 'Arnavutköy']
['Hastane', 'Arnavutköy']
['Hicret', 'Arnavutköy']
['Karaburun', 'Arnavutköy']
['Karlıbayır', 'Arnavutköy']
['Mavigöl', 'Arnavutköy']
['Mustafa Kemal Paşa', 'Arnavutköy']
['Nenehatun', 'Arnavutköy']
['Sazlıbosna', 'Arnavutköy']
['Tayakadın', 'Arnavutköy']
['Taşoluk', 'Arnavutköy']
['Taşoluk Adnan Menderes', 'Arnavutköy']
['Taşoluk Fatih', 'Arnavutköy']
['Taşoluk M. Fevzi Çakmak', 'Arnavutköy']
['Taşoluk Mehmet Akif Ersoy', 'Arnavutköy']
['Taşoluk Çilingir', 'Arnavutk

['Bağlarçeşme', 'Esenyurt']
['Cumhuriyet', 'Esenyurt']
['Esenkent', 'Esenyurt']
['Fatih', 'Esenyurt']
['Güzelyurt (Haramidere)', 'Esenyurt']
['Hürriyet', 'Esenyurt']
['Koza', 'Esenyurt']
['Mehmet Akif Ersoy', 'Esenyurt']
['Mehterçeşme', 'Esenyurt']
['Mevlana', 'Esenyurt']
['Namik Kemal', 'Esenyurt']
['Necip Fazıl Kısakürek', 'Esenyurt']
['Orhan Gazi', 'Esenyurt']
['Osmangazi', 'Esenyurt']
['Piri Reis', 'Esenyurt']
['Pınar', 'Esenyurt']
['Saadetdere', 'Esenyurt']
['Selahaddin Eyyubi', 'Esenyurt']
['Sultaniye', 'Esenyurt']
['Süleymaniye', 'Esenyurt']
['Talatpasa', 'Esenyurt']
['Turgut Özal', 'Esenyurt']
['Yenikent', 'Esenyurt']
['Yeşilkent', 'Esenyurt']
['Yunus Emre', 'Esenyurt']
['Zafer', 'Esenyurt']
['Çınar', 'Esenyurt']
['Örnek', 'Esenyurt']
['Üçevler', 'Esenyurt']
['İncirtepe', 'Esenyurt']
['İnönü', 'Esenyurt']
['İstiklal', 'Esenyurt']
['Şehitler', 'Esenyurt']
['Akpınar', 'Eyüp']
['Akşemsettin', 'Eyüp']
['Alibeyköy', 'Eyüp']
['Ağaçlı', 'Eyüp']
['Defterdar', 'Eyüp']
['Düğmeciler', 'Ey

['PTT Evleri', 'Sarıyer']
['Poligon', 'Sarıyer']
['Pınar', 'Sarıyer']
['Reşitpaşa', 'Sarıyer']
['Rumelifeneri', 'Sarıyer']
['Rumelihisarı', 'Sarıyer']
['Rumelikavağı', 'Sarıyer']
['Tarabya', 'Sarıyer']
['Uskumruköy', 'Sarıyer']
['Yeni Mahalle', 'Sarıyer']
['Yeniköy', 'Sarıyer']
['Zekeriyaköy', 'Sarıyer']
['Çayırbaşı', 'Sarıyer']
['İstinye', 'Sarıyer']
['Akören', 'Silivri']
['Alibey', 'Silivri']
['Alipaşa', 'Silivri']
['Balaban', 'Silivri']
['Bekirli', 'Silivri']
['Beyciler', 'Silivri']
['Büyük Kılıçlı', 'Silivri']
['Büyük Sinekli', 'Silivri']
['Büyük Çavuşlu', 'Silivri']
['Cumhuriyet', 'Silivri']
['Danamandıra', 'Silivri']
['Fatih', 'Silivri']
['Fener', 'Silivri']
['Fevzipaşa', 'Silivri']
['Gazitepe', 'Silivri']
['Gümüşyaka', 'Silivri']
['Hürriyet', 'Silivri']
['Kadıköy', 'Silivri']
['Kavaklı', 'Silivri']
['Kurfallı', 'Silivri']
['Küçük Kılıçlı', 'Silivri']
['Küçük Sinekli', 'Silivri']
['Mimar Sinan', 'Silivri']
['Ortaköy', 'Silivri']
['Piri Mehmet Paşa', 'Silivri']
['Sancaktepe', 'Sil

In [282]:
datahead = ["Neighborhood",
            "County", 
            "ReportName",
            "AvgSalePrice", 
            "MinSalePrice", 
            "MaxSalePrice", 
            "AvgRentPrice", 
            "MinRentPrice", 
            "MaxRentPrice",
            "ReturnPeriod",
            "SalePrice1MonthChange",
            "SalePrice3YearChange",
            "SalePrice5YearChange",
            "RentPrice1MonthChange",
            "RentPrice3YearChange",
            "RentPrice5YearChange",
            "Population",
            "SocioeconomicClas",
            "MarriedPerc",
            "SinglePerc"]

dfstats = pd.DataFrame(data, columns = datahead)
dfstats.loc[dfstats.ReportName=="İstanbul Bölge Raporu",dfstats.columns[2:]] = np.nan

for col in ["AvgSalePrice", "MinSalePrice", "MaxSalePrice", 
            "AvgRentPrice", "MinRentPrice", "MaxRentPrice"]:
    dfstats[col] = dfstats[col].str.replace("TL", "").str.replace(".","").str.strip().astype(float)
    
for col in ["SalePrice1MonthChange", "SalePrice3YearChange", "SalePrice5YearChange",
            "RentPrice1MonthChange", "RentPrice3YearChange", "RentPrice5YearChange"]:
    dfstats[col] = dfstats[col].str.replace("%", "").astype("float") / 100

dfstats["ReturnPeriod"] = dfstats["ReturnPeriod"].str.replace("YIL", "").str.strip().astype(float)
dfstats["Population"] = dfstats["Population"].str.replace(".", "").astype(float)
dfstats["MarriedPerc"] = dfstats["MarriedPerc"].str.replace("Evli", "").str.replace("%","").astype(float)
dfstats["SinglePerc"] = dfstats["SinglePerc"].str.replace("Bekar", "").str.replace("%","").astype(float)

dfstats.to_pickle(os.getcwd()+"/data/IstanbulSocioEconomics.pkl")

In [295]:
dfstats = pd.read_pickle(os.getcwd()+"/data/IstanbulSocioEconomics.pkl")
dfstats

,Neighborhood,County,ReportName,AvgSalePrice,MinSalePrice,MaxSalePrice,AvgRentPrice,MinRentPrice,MaxRentPrice,ReturnPeriod,SalePrice1MonthChange,SalePrice3YearChange,SalePrice5YearChange,RentPrice1MonthChange,RentPrice3YearChange,RentPrice5YearChange,Population,SocioeconomicClas,MarriedPerc,SinglePerc
0,Anadolu,Arnavutköy,"Anadolu, Arnavutköy, İstanbul Bölge Raporu",247932.0,185949.0,309915.0,925.0,694.0,1156.0,22.0,0.0214,0.274,0.480,0.0199,0.231,0.362,38263.0,C-,67.0,27.0
1,Arnavutköy Merkez,Arnavutköy,"Arnavutköy Merkez, Arnavutköy, İstanbul Bölge ...",265134.0,198851.0,331418.0,1083.0,812.0,1354.0,20.0,0.0075,0.247,0.712,0.0298,0.362,0.715,20273.0,B-,66.0,26.0
2,Arnavutköy Yavuzselim,Arnavutköy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arnavutköy İmrahor,Arnavutköy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arnavutköy İslambey,Arnavutköy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
949,Paşa,Şişli,"Paşa, Şişli, İstanbul Bölge Raporu",462719.0,347039.0,578399.0,2312.0,1734.0,2890.0,17.0,-0.0010,0.071,0.205,0.0041,0.177,0.254,19365.0,B+,55.0,31.0
950,Teşvikiye,Şişli,"Teşvikiye, Şişli, İstanbul Bölge Raporu",1324490.0,993368.0,1655613.0,4151.0,3113.0,5189.0,27.0,0.0327,-0.046,0.225,-0.0197,0.162,0.093,11506.0,A+,47.0,30.0
951,Yayla,Şişli,"Yayla, Şişli, İstanbul Bölge Raporu",571858.0,428894.0,714823.0,3114.0,2336.0,3893.0,15.0,-0.0304,-0.109,-0.018,0.0331,0.083,0.118,11520.0,A,57.0,32.0
952,İnönü,Şişli,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [292]:
dfstats[dfstats.County=="Beykoz"]

,Neighborhood,County,ReportName,AvgSalePrice,MinSalePrice,MaxSalePrice,AvgRentPrice,MinRentPrice,MaxRentPrice,ReturnPeriod,SalePrice1MonthChange,SalePrice3YearChange,SalePrice5YearChange,RentPrice1MonthChange,RentPrice3YearChange,RentPrice5YearChange,Population,SocioeconomicClas,MarriedPerc,SinglePerc
134,Acarlar,Beykoz,"Acarlar, Beykoz, İstanbul Bölge Raporu",1297471.0,973103.0,1621839.0,4087.0,3065.0,5109.0,26.0,0.0330,0.295,1.114,-0.0094,0.220,0.792,7427.0,A+,62.0,27.0
135,Akbaba,Beykoz,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
136,Alibahadır,Beykoz,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
137,Anadolufeneri,Beykoz,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
138,Anadoluhisarı,Beykoz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139,Anadolukavağı,Beykoz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140,Baklacı,Beykoz,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
141,Bozhane,Beykoz,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
142,Cumhuriyet,Beykoz,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
143,Dereseki,Beykoz,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN


In [294]:
df[df.County=="Beykoz"]

,Neighborhood,County,Area,Search
134,Acarlar,Beykoz,5.4,beykoz-acarlar
135,Akbaba,Beykoz,4.6,beykoz-akbaba
136,Alibahadır,Beykoz,11.2,beykoz-alibahadir
137,Anadolufeneri,Beykoz,17.0,beykoz-anadolufeneri
138,Anadoluhisarı,Beykoz,0.99,beykoz-anadoluhisari
139,Anadolukavağı,Beykoz,5.2,beykoz-anadolukavagi
140,Baklacı,Beykoz,11.8,beykoz-baklaci
141,Bozhane,Beykoz,14.3,beykoz-bozhane
142,Cumhuriyet,Beykoz,10.6,beykoz-cumhuriyet
143,Dereseki,Beykoz,8.6,beykoz-dereseki
